In [ ]:
!pip install sacrebleu
!pip install happytransformer

In [3]:
from happytransformer import HappyTextToText
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
import pandas as pd
import torch
from datasets import load_dataset
import csv
from datasets import load_dataset, load_metric

In [4]:
model_path = '/content/drive/MyDrive/Google Colab/Advanced NLP/GEC_C4_model/'
T5 = HappyTextToText(load_path=model_path)

04/04/2022 11:08:47 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [5]:
def check_sample(sample):
    for correction in sample["corrections"]:
        if not (sample['sentence'] and correction):
            return False
    return True

In [ ]:
eval_dataset = load_dataset("jfleg", split='test[:]')

incorrect_samples = []
correct_refs = [[],[],[],[]]

for sample in eval_dataset:
    if check_sample(sample)==True:
        incorrect_samples.append(sample['sentence'])
        for i in range(len(sample['corrections'])):
            correct_refs[i].append(sample['corrections'][i])

In [ ]:
print(incorrect_samples[1])
for i in range(4):
    print(correct_refs[i][1])

In [8]:
beam_settings = TTSettings(num_beams=5, min_length=1, max_length=256)
hyps = []
cnt = 0

for sample in incorrect_samples:
    result = T5.generate_text('grammar: ' + sample, args=beam_settings)
    hyps.append(result.text)
    cnt+=1
    if cnt%100==0:
        print(cnt)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:999: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


100
200
300
400
500
600
700


In [ ]:
from sacrebleu.metrics import BLEU
bleu = BLEU()
result = bleu.corpus_score(hyps, correct_refs)
print(result)

In [ ]:
from sacrebleu.metrics import BLEU

refs = [['The dog bit the guy.', 'It was not unexpected.', 'The man bit him first.'],
        ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.']]

hyps = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

bleu = BLEU()

result = bleu.corpus_score(hyps, refs)
print(result)